START TO REGRESSSION MODEL CREATION

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression



In [2]:
df = pd.read_csv("cleaned_data.csv")
df

,name,company,year,Price,kms_driven,fuel_type,Age,Annual_Km_Driven
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,80000,45000,Petrol,13,3461.538462
1,Mahindra Jeep CL550 MDI,Mahindra,2006,425000,40,Diesel,14,2.857143
2,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,325000,28000,Petrol,6,4666.666667
3,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,575000,36000,Diesel,6,6000.000000
4,Ford Figo,Ford,2012,175000,41000,Diesel,8,5125.000000
...,...,...,...,...,...,...,...,...
811,Maruti Suzuki Ritz VXI ABS,Maruti,2011,270000,50000,Petrol,9,5555.555556
812,Tata Indica V2 DLE BS III,Tata,2009,110000,30000,Diesel,11,2727.272727
813,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol,11,12000.000000
814,Tata Zest XM Diesel,Tata,2018,260000,27000,Diesel,2,13500.000000


In [3]:
X = df[['year', 'kms_driven']] 
y = df['Price'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0)


In [4]:
model = LinearRegression()  
model.fit(X_train, y_train)  


LinearRegression()

In [5]:
y_pred = model.predict(X_test) 


In [6]:
model.predict([[2015,25000]])

d:\Users\mohit\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([504085.5048441])

In [7]:
model.predict([[2020,20000]])

d:\Users\mohit\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([666265.61238368])

In [8]:
model.predict([[2019,52000]])

d:\Users\mohit\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([607438.07537043])